In [161]:
import pandas as pd
import re

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
dutch_stopwords = stopwords.words('dutch')

In [162]:
# Read the preprocessed data
df = pd.read_csv('a:/df_cleaned.csv')
df.head()


# df = pd.read_csv('a:/test-data/bloeding-met-patientenlijst-copy-4-verslagen.csv')
# df['alltext'] = df['verslagen_report_content']
# df['date'] = pd.to_datetime(df['verslagen_report_start_date'], errors='coerce')
# df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street..."
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob..."
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ..."
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'..."
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee..."


In [163]:
# Define regex pattern

# terms = [
#     # r"maagdarmbloeding", r"gastro[-\s]?intestinale bloeding",
#     # r" gib ", r" agib ", r"gi\s*bloeding",

#     r"bloedbraken", r"bloed\s*braken", r"hematemesis", r"melena",
#     r"maagdarmbloeding", r"gastro[-\s]?intestinale bloeding"
#     r"rectaal\s*bloedverlie", r"rectaal\s*bloedverlies", 
#     r"vers\s*bloed\s*per\s*anum",
#     r"bloeding\s*uit\s*het\s*maag[-\s]?darmkanaal",
# ]


terms = [
    r"bloedbraken", r"bloed\s*braken", r"hematemesis", r"melena",
    r"maagdarmbloeding", r"gastro[-\s]?intestinale bloeding",
    r"rectaal\s*bloedverlies?",  # aangepast van bloedverlies naar bloedverlie
    r"vers\s*bloed\s*per\s*anum",
    r"bloeding\s*uit\s*het\s*maag[-\s]?darmkanaal",
    r" gib ", r" agib ", r"gi\s*bloeding",
    r"anemie"
]

terms = [
    r"acute gi\s*bloeding", r"acute bloeding",
    r"acute maagdarmbloeding", r"acute gastro[-\s]?intestinale bloeding",
    r" agib "
    ]

#acute te algemeen. acute buikpijn, acute pancreatitis

# Escape multi-word phrases properly
pattern_str = "|".join([fr"\b{t}\b" if " " not in t and "[- ]" not in t else t for t in terms])
agib_pattern = re.compile(pattern_str, re.IGNORECASE)



def extract_context(text, pattern, window=5):
    matches = []
    for match in pattern.finditer(text):
        start, end = match.start(), match.end()
        before = text[:start]
        after = text[end:]

        # Get words before match
        before_words = re.findall(r'\w+', before)[-window:]
        # Get words after match
        after_words = re.findall(r'\w+', after)[:window]

        context = ' '.join(before_words + [match.group()] + after_words)
        matches.append(context)

    return matches[0] if matches else None

# Apply function
df['agib_context'] = df['alltext'].apply(lambda x: extract_context(str(x), agib_pattern))

# Function to extract the exact match
def extract_match(text, pattern):
    match = pattern.search(text)
    return match.group() if match else None

# Apply function to extract the exact match
df['agib_match'] = df['alltext'].apply(lambda x: extract_match(str(x), agib_pattern))
df.head()



,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_context,agib_match
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",None,None
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob...",None,None
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ...",None,None
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'...",None,None
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee...",None,None


In [164]:
# def extract_relevant_sections(text, relevant_headers=None):
#     if relevant_headers is None:
#         relevant_headers = ["anamnese", "conclusie", "beleid", "tractusanamnese", "triagekeuze"]

#     # Split op kopjes
#     sections = re.split(r"(?<=\n)([A-Z][a-z]+(?:\s*[A-Z][a-z]+)*):", text)
    
#     # Combine sections into dict: {header: content}
#     it = iter(sections)
#     section_dict = {}
#     current = next(it, None)
#     while current:
#         header = current.strip().lower()
#         content = next(it, "")
#         section_dict[header] = content.strip()
#         current = next(it, None)

#     # Extract relevant parts only
#     combined = ""
#     for header in relevant_headers:
#         if header in section_dict:
#             combined += section_dict[header] + "\n"
    
#     return combined


In [165]:
def extract_context(text, pattern, window=5):
    matches = []
    for match in pattern.finditer(text):
        start, end = match.start(), match.end()
        before = text[:start]
        after = text[end:]

        # Get words before match
        before_words = re.findall(r'\w+', before)[-window:]
        # Get words after match
        after_words = re.findall(r'\w+', after)[:window]

        context = ' '.join(before_words + [match.group()] + after_words)
        matches.append(context)

    return matches[0] if matches else None





In [166]:
# Apply regex
df['agib_mention'] = df['alltext'].apply(lambda x: bool(agib_pattern.search(str(x))))

# Optionally exclude negated matches
negation_pattern = re.compile(r"\b(geen|niet|zonder)\b.{0,20}?\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)
df['negated'] = df['alltext'].apply(lambda x: bool(negation_pattern.search(str(x))))
df['agib_final'] = df['agib_mention'] & ~df['negated']


# Apply function
df['agib_context'] = df['alltext'].apply(lambda x: extract_context(str(x), agib_pattern))
df.head()
# Function to extract the exact match
def extract_match(text, pattern):
    match = pattern.search(text)
    return match.group() if match else None

# Apply function to extract the exact match
df['agib_match'] = df['alltext'].apply(lambda x: extract_match(str(x), agib_pattern))
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_context,agib_match,agib_mention,negated,agib_final
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",None,None,False,False,False
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob...",None,None,False,False,False
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ...",None,None,False,False,False
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'...",None,None,False,False,False
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee...",None,None,False,False,False


In [167]:
# # Nieuwe kolom met alleen relevante tekst
# df['agib_relevant_text'] = df['alltext'].apply(lambda x: extract_relevant_sections(str(x)))

# # Match alleen in relevante tekst
# df['agib_mention_relevant'] = df['agib_relevant_text'].apply(lambda x: bool(agib_pattern.search(x)))

# # Context + match optioneel:
# df['agib_match'] = df['agib_relevant_text'].apply(lambda x: extract_match(str(x), agib_pattern))
# df['agib_context'] = df['agib_relevant_text'].apply(lambda x: extract_context(str(x), agib_pattern))


In [168]:
# # Apply regex
# df['agib_mention'] = df['alltext'].apply(lambda x: bool(agib_pattern.search(str(x))))

# # Optionally exclude negated matches
# negation_pattern = re.compile(r"\b(geen|niet|zonder)\b.{0,20}?\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)
# df['negated'] = df['alltext'].apply(lambda x: bool(negation_pattern.search(str(x))))
# df['agib_final'] = df['agib_mention'] & ~df['negated']

In [169]:
def extract_context(text, pattern, window=5):
    matches = []
    for match in pattern.finditer(text):
        start, end = match.start(), match.end()
        before = text[:start]
        after = text[end:]

        # Get words before match
        before_words = re.findall(r'\w+', before)[-window:]
        # Get words after match
        after_words = re.findall(r'\w+', after)[:window]

        context = ' '.join(before_words + [match.group()] + after_words)
        matches.append(context)

    return matches[0] if matches else None



# Apply function
df['agib_context'] = df['alltext'].apply(lambda x: extract_context(str(x), agib_pattern))
df.head()
# Function to extract the exact match
def extract_match(text, pattern):
    match = pattern.search(text)
    return match.group() if match else None

# Apply function to extract the exact match
df['agib_match'] = df['alltext'].apply(lambda x: extract_match(str(x), agib_pattern))
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_context,agib_match,agib_mention,negated,agib_final
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",None,None,False,False,False
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob...",None,None,False,False,False
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ...",None,None,False,False,False
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'...",None,None,False,False,False
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee...",None,None,False,False,False


In [170]:
print("Aantal agib_mention == True:", df['agib_mention'].sum())


Aantal agib_mention == True: 25


In [171]:
print("Aantal negated == True:", df['negated'].sum())

Aantal negated == True: 0


In [172]:
for i, row in df[df['agib_match'].notnull()].iterrows():
    print(f"🩸 Date: {row['date']}")
    print(f"🩸 Match: {row['agib_match']}")
    print(f"📍 Context: {row['agib_context']}")
    print(f"📜 Content: {row['verslagen_report_content']}")
    print("-----")


🩸 Date: 2021-06-24 15:27:00
🩸 Match: acute bloeding
📍 Context: samenvatting acute bloeding dun darm vce Clopidogrel gastrojejunoscopie
📜 Content: Samenvatting: 
Acute bloeding in dunne darm bij VCE onder clopidogrel. Gastro-jejunoscopie + ileocoloscopie: gb. DBE tot 200cm voorbij Treitz in ETZ onder afwijkingen.
MRI: afwijkingen onderbuik; massa rechter adnex wv icc gyn loopt.
VG hemolytische anemie, mamma ca, TIA, clopidogrel (tijdelijk gestopt).


Beloop: 
onderzoek gaat niet door, pte belt dit door.
-----
🩸 Date: 2021-06-03 11:31:00
🩸 Match: acute bloeding
📍 Context: samenvatting acute bloeding dun darm vce Clopidogrel gastrojejunoscopie
📜 Content: Samenvatting: 
Acute bloeding in dunne darm bij VCE onder clopidogrel. Gastro-jejunoscopie + ileocoloscopie: gb. DBE tot 200cm voorbij Treitz in ETZ onder afwijkingen.
MRI: afwijkingen onderbuik; massa rechter adnex wv icc gyn loopt.
VG hemolytische anemie, mamma ca, TIA, clopidogrel (tijdelijk gestopt).


Beloop: 
Telefonisch i.v.m. COVI

In [173]:
# for i, row in df[df['pseudo_id'] == 'C71B153E9F184C29E8D39654E41C8B54586E3AF4'].iterrows():
#     print(f"🩸 Date: {row['date']}")
#     print(f"🩸 Match: {row['agib_match']}")
#     print(f"📍 Context: {row['agib_context']}")
#     print(f"📜 Content: {row['verslagen_report_content']}")
#     print("-----")


In [174]:
pseudo_ids_with_agib = df[df['agib_mention'] == True]['pseudo_id'].unique()
pseudo_ids_df = pd.DataFrame(pseudo_ids_with_agib, columns=['pseudo_id'])
pseudo_ids_df

,pseudo_id
0,BEBEFC726C577A7B40F1A467F4E59746FBC7F76B
1,CF58B7215ED673FD2AC116C49953A9941E73F597
2,FD8C682C1F4FDA1E5EC0B760D30875556419BD71


In [175]:
# all_ids = df['pseudo_id'].unique()
all_ids_df = pd.DataFrame(all_ids, columns=['pseudo_id'])
all_ids_df

,pseudo_id
0,0001A243648CFB955C3718FD1CE81276D1F4F260
1,0003C64B626E17E45309A443C70019E76AB0C407
2,00067EE4483868D5C3EEA1E072648CA61959714D
3,0006C6C74E4AC9C249EBE1F33DAE8EFCCBFBA783
4,0007089F5A45B8DA493CB883F40336EF4CC3254D
...,...
3418,F797097BA39F81B05EEC3F251973743CA03CB9B4
3419,F8C241E69061BB450778B7A4CC336FF520469A0A
3420,F8D116F01EE0039678998F393FA337C10AD4F4E2
3421,FAA79717FF2C725767E9469350ACECF640E5FCBC


In [176]:
df_dbc = pd.read_csv('a:/test-data/bloeding-met-patientenlijst-copy-2-diagnose-db-cs.csv')
print(len(df_dbc)), df_dbc.head()

2522


(None,
                                   pseudo_id  \
 0  0021F09BD5BB8FEFE4BD37362731099019AA6EB5   
 1  0021F09BD5BB8FEFE4BD37362731099019AA6EB5   
 2  0021F09BD5BB8FEFE4BD37362731099019AA6EB5   
 3  002F7F499E7875932FB6F0DCE675B843AFB507A7   
 4  002F7F499E7875932FB6F0DCE675B843AFB507A7   
 
    diagnose_db_cs_dbc_specialty_code  \
 0                                318   
 1                                318   
 2                                318   
 3                                318   
 4                                318   
 
     diagnose_db_cs_dbc_specialty_code_description  \
 0  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 1  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 2  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 3  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 4  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 
    diagnose_db_cs_dbc_diagnosis_code  \
 0                                612   
 1                                612   
 2    

In [177]:
# Merge df with df_dbc on pseudo_id
df = df.merge(df_dbc[['pseudo_id', 'diagnose_db_cs_dbc_diagnosis_code']], on='pseudo_id', how='left')

# Display the updated dataframe
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_context,agib_match,agib_mention,negated,agib_final,diagnose_db_cs_dbc_diagnosis_code
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",None,None,False,False,False,203
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",None,None,False,False,False,203
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",None,None,False,False,False,203
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",None,None,False,False,False,203
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",None,None,False,False,False,612


In [178]:
df['diagnose_db_cs_dbc_diagnosis_code'].unique()

array([203, 612], dtype=int64)

In [179]:
# Filter rows where agib_final is True and check for missing diagnosis codes
missing_diagnosis = df[df['agib_final'] & df['diagnose_db_cs_dbc_diagnosis_code'].isnull()]

# Display the result
if missing_diagnosis.empty:
    print("All rows with agib_final == True have a diagnose_db_cs_dbc_diagnosis_code.")
else:
    print("There are rows with agib_final == True but no diagnose_db_cs_dbc_diagnosis_code:")
    print(missing_diagnosis)

All rows with agib_final == True have a diagnose_db_cs_dbc_diagnosis_code.


In [180]:
missing_diagnosis

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_context,agib_match,agib_mention,negated,agib_final,diagnose_db_cs_dbc_diagnosis_code


In [181]:
pseudo_ids_with_agib = df[df['agib_mention'] == True]['pseudo_id'].unique()
pseudo_ids_df = pd.DataFrame(pseudo_ids_with_agib, columns=['pseudo_id', 'diagnose_db_cs_dbc_diagnosis_code'])
pseudo_ids_df

ValueError: Shape of passed values is (3, 1), indices imply (3, 2)